In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
import os
import tensorflow as tf
import numpy as np
import urllib.request as req
import shutil
from tqdm import tqdm
from PIL import Image

In [4]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

In [3]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file="data.zip"
DESTINATION_ZIP_PATH = os.path.join(DESTINATION,data_file)

In [4]:
DESTINATION_ZIP_PATH

'data\\data.zip'

In [5]:
pwd

'C:\\ds\\projects\\dl_projects\\CNN-classifier-fsdsf21\\research_env'

In [8]:
filename,headers = tqdm(req.urlretrieve(SOURCE_URL,DESTINATION_ZIP_PATH))

100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


In [6]:
filename

NameError: name 'filename' is not defined

In [10]:
print(headers)

Content-Type: application/octet-stream
Accept-Ranges: bytes
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-blob-content-md5: 4TekUHNw2UJGm20meiTqBA==
Last-Modified: Mon, 09 May 2022 20:42:07 GMT
ETag: "0x8DA31FC621E01C4"
Content-Length: 824887076
Date: Sat, 25 Feb 2023 12:25:29 GMT
Connection: close




In [11]:
from zipfile import ZipFile
with ZipFile(filename,'r') as zip_file:
    zip_file.extractall("data/")

In [7]:
TARGET_DATA_DIR = ["Cat","Dog"]
#PARENT_DIR ="data/PetImages" for linux
PARENT_DIR =os.path.join("data","PetImages") # for all os

In [8]:
for dirs in os.listdir(PARENT_DIR):
    print(dirs)

Cat
Dog


In [9]:
for dirs in os.listdir(PARENT_DIR):
    print(os.path.join(PARENT_DIR,dirs))

data\PetImages\Cat
data\PetImages\Dog


In [15]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        print(dirs,imgs)
        

Cat 0.jpg
Cat 1.jpg
Cat 10.jpg
Cat 100.jpg
Cat 1000.jpg
Cat 10000.jpg
Cat 10001.jpg
Cat 10002.jpg
Cat 10003.jpg
Cat 10004.jpg
Cat 10005.jpg
Cat 10006.jpg
Cat 10007.jpg
Cat 10008.jpg
Cat 10009.jpg
Cat 1001.jpg
Cat 10010.jpg
Cat 10011.jpg
Cat 10012.jpg
Cat 10013.jpg
Cat 10014.jpg
Cat 10015.jpg
Cat 10016.jpg
Cat 10017.jpg
Cat 10018.jpg
Cat 10019.jpg
Cat 1002.jpg
Cat 10020.jpg
Cat 10021.jpg
Cat 10022.jpg
Cat 10023.jpg
Cat 10024.jpg
Cat 10025.jpg
Cat 10026.jpg
Cat 10027.jpg
Cat 10028.jpg
Cat 10029.jpg
Cat 1003.jpg
Cat 10030.jpg
Cat 10031.jpg
Cat 10032.jpg
Cat 10033.jpg
Cat 10034.jpg
Cat 10035.jpg
Cat 10036.jpg
Cat 10037.jpg
Cat 10038.jpg
Cat 10039.jpg
Cat 1004.jpg
Cat 10040.jpg
Cat 10041.jpg
Cat 10042.jpg
Cat 10043.jpg
Cat 10044.jpg
Cat 10045.jpg
Cat 10046.jpg
Cat 10047.jpg
Cat 10048.jpg
Cat 10049.jpg
Cat 1005.jpg
Cat 10050.jpg
Cat 10051.jpg
Cat 10052.jpg
Cat 10053.jpg
Cat 10054.jpg
Cat 10055.jpg
Cat 10056.jpg
Cat 10057.jpg
Cat 10058.jpg
Cat 10059.jpg
Cat 1006.jpg
Cat 10060.jpg
Cat 10061.jp

Dog 3867.jpg
Dog 3868.jpg
Dog 3869.jpg
Dog 387.jpg
Dog 3870.jpg
Dog 3871.jpg
Dog 3872.jpg
Dog 3873.jpg
Dog 3874.jpg
Dog 3875.jpg
Dog 3876.jpg
Dog 3877.jpg
Dog 3878.jpg
Dog 3879.jpg
Dog 388.jpg
Dog 3880.jpg
Dog 3881.jpg
Dog 3882.jpg
Dog 3883.jpg
Dog 3884.jpg
Dog 3885.jpg
Dog 3886.jpg
Dog 3887.jpg
Dog 3888.jpg
Dog 3889.jpg
Dog 389.jpg
Dog 3890.jpg
Dog 3891.jpg
Dog 3892.jpg
Dog 3893.jpg
Dog 3894.jpg
Dog 3895.jpg
Dog 3896.jpg
Dog 3897.jpg
Dog 3898.jpg
Dog 3899.jpg
Dog 39.jpg
Dog 390.jpg
Dog 3900.jpg
Dog 3901.jpg
Dog 3902.jpg
Dog 3903.jpg
Dog 3904.jpg
Dog 3905.jpg
Dog 3906.jpg
Dog 3907.jpg
Dog 3908.jpg
Dog 3909.jpg
Dog 391.jpg
Dog 3910.jpg
Dog 3911.jpg
Dog 3912.jpg
Dog 3913.jpg
Dog 3914.jpg
Dog 3915.jpg
Dog 3916.jpg
Dog 3917.jpg
Dog 3918.jpg
Dog 3919.jpg
Dog 392.jpg
Dog 3920.jpg
Dog 3921.jpg
Dog 3922.jpg
Dog 3923.jpg
Dog 3924.jpg
Dog 3925.jpg
Dog 3926.jpg
Dog 3927.jpg
Dog 3928.jpg
Dog 3929.jpg
Dog 393.jpg
Dog 3930.jpg
Dog 3931.jpg
Dog 3932.jpg
Dog 3933.jpg
Dog 3934.jpg
Dog 3935.jpg
Dog 3936

In [10]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        print(os.path.join(full_path_data_dir,imgs))

data\PetImages\Cat\0.jpg
data\PetImages\Cat\1.jpg
data\PetImages\Cat\10.jpg
data\PetImages\Cat\100.jpg
data\PetImages\Cat\1000.jpg
data\PetImages\Cat\10000.jpg
data\PetImages\Cat\10001.jpg
data\PetImages\Cat\10002.jpg
data\PetImages\Cat\10003.jpg
data\PetImages\Cat\10004.jpg
data\PetImages\Cat\10005.jpg
data\PetImages\Cat\10006.jpg
data\PetImages\Cat\10007.jpg
data\PetImages\Cat\10008.jpg
data\PetImages\Cat\10009.jpg
data\PetImages\Cat\1001.jpg
data\PetImages\Cat\10010.jpg
data\PetImages\Cat\10011.jpg
data\PetImages\Cat\10012.jpg
data\PetImages\Cat\10013.jpg
data\PetImages\Cat\10014.jpg
data\PetImages\Cat\10015.jpg
data\PetImages\Cat\10016.jpg
data\PetImages\Cat\10017.jpg
data\PetImages\Cat\10018.jpg
data\PetImages\Cat\10019.jpg
data\PetImages\Cat\1002.jpg
data\PetImages\Cat\10020.jpg
data\PetImages\Cat\10021.jpg
data\PetImages\Cat\10022.jpg
data\PetImages\Cat\10023.jpg
data\PetImages\Cat\10024.jpg
data\PetImages\Cat\10025.jpg
data\PetImages\Cat\10026.jpg
data\PetImages\Cat\10027.jpg
d

In [11]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir,imgs)
        try:
            img=Image.open(path_to_img)
            img.verify()
            #print(f"{path_to_img } is verified")
        
        except Exception as e:
            print(f"{path_to_img } is bad")
            
            

data\PetImages\Dog\11702.jpg is bad
data\PetImages\Dog\Thumbs.db is bad


C:\ds\projects\dl_projects\CNN-classifier-fsdsf21\env\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [12]:
BAD_DATA_DIR = "bad_data"
os.makedirs(BAD_DATA_DIR,exist_ok=True)

In [13]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR,dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir,imgs)
        try:
            img=Image.open(path_to_img)
            img.verify()
            #print(f"{path_to_img } is verified")
        
        except Exception as e:
            print(f"{path_to_img } is bad")
            bad_data_path=os.path.join(BAD_DATA_DIR,imgs)
            shutil.move(path_to_img,bad_data_path)
            
            

data\PetImages\Dog\11702.jpg is bad
data\PetImages\Dog\Thumbs.db is bad


In [14]:
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

In [15]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 24998 files belonging to 2 classes.
Using 19999 files for training.
Found 24998 files belonging to 2 classes.
Using 4999 files for validation.


In [16]:
### Create a log dir for tensorboard logs

import time

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName = time.asctime().replace(" ", "_").replace(":", "")
    log_path = os.path.join(base_log_dir, uniqueName)
    print(f"saving logs at: {log_path}")
    return log_path

log_dir = get_log_path()

saving logs at: logs\fit\Sat_Feb_25_130616_2023


In [17]:
file_writer = tf.summary.create_file_writer(logdir=log_dir)

In [18]:
for images, labels in train_ds.take(1):
    print(images.shape, labels)

(32, 180, 180, 3) tf.Tensor([0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 1 1], shape=(32,), dtype=int32)


In [19]:
images.shape

TensorShape([32, 180, 180, 3])

with file_writer.as_default():
    images = np.reshape(images[10:30],(-1, 180, 180, 3)) ### (20, 28, 28, 1)

    tf.summary.image("samples",images, max_outputs=25, step=0)

In [66]:
#%load_ext tensorboard
#%tensorboard --logdir=research_env/logs/fit

In [58]:
#%tensorboard --logdir=research_env/logs/fit

In [20]:
with file_writer.as_default():
    images = np.array(images) ### (20, 28, 28, 1)

    tf.summary.image("samples",images.astype("uint8"), max_outputs=25, step=0)

In [23]:
%reload_ext tensorboard

In [24]:
%load_ext tensorboard
%tensorboard --logdir=research_env/logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2504), started 0:33:44 ago. (Use '!kill 2504' to kill it.)

In [72]:
%reload_ext tensorboard

In [84]:
!kill 2504

kill: 2504: No such process
